# Information regarding the data that this notebook refers to

In [1]:
# fill out the following fields. Check that input data and notebook name match.

# reference vehicle_fleet_data.xlsx notebook metadata to select notebook_id
notebook_id= '004'
name_string= '_s_EU_2012_2018_cl(pc)_me_eurostat'

nb_name= notebook_id + name_string +'.ipynb'
nb_input_workbook= 'in' + name_string + '.xlsx'
nb_output_workbook= notebook_id + '.xlsx'
nb_stock_or_flow= 'stock'
nb_geography= '150'
nb_start_time= '2012'
nb_stop_time= '2018'
nb_attribute_1= 'passenger cars'
nb_attribute_2= 'registered'
nb_attribute_3= 'motor energy'
nb_attribute_4= ''
nb_data_source= 'EUROSTAT'
nb_data_source_url= 'https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=road_eqs_carpda&lang=en'
nb_comment= ''

In [2]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib as plt

In [3]:
# write the notbook metadata into a data frame
notebook_metadata = pd.DataFrame({'notebook_name':nb_name,
                                 'input_file': nb_input_workbook, 
                                 'output_file': nb_output_workbook, 
                                 'source': nb_data_source, 
                                 'geography': nb_geography,
                                 'start_time': nb_start_time,
                                 'stop_time': nb_stop_time, 
                                 'attribute1': nb_attribute_1, 
                                 'attribute2': nb_attribute_2,
                                 'attribute3': nb_attribute_3,
                                 'attribute4': nb_attribute_4,
                                 'source_url' : nb_data_source_url,
                                 'comment': nb_comment}, index=[notebook_id])

## Reading in the excel data and merging the sheets into one dataframe with category info attached to row data

In [4]:
nb_input_workbook

'in_s_EU_2012_2018_cl(pc)_me_eurostat.xlsx'

In [5]:
# read in data,
xls = pd.ExcelFile(nb_input_workbook)

In [6]:
# extract the data into a dataframe or several
df = pd.read_excel(xls)
df.head(2)

,TIME,MOT_NRG,GEO,UNIT,Value,Flag and Footnotes
0,2012,Total,Belgium,NR,5444000,NaN
1,2012,Total,Bulgaria,NR,2807000,NaN


## Import dictionary for country codes and replace country names with 3 digit codes

In [7]:
# rename region to 'geo'
df.rename(columns={'GEO':'geo'},inplace= True)
df.rename(columns={'Flag and Footnotes':'footnote'},inplace= True)
df.rename(columns={'MOT_NRG':'motor_energy'},inplace= True)
df.rename(columns={'TIME':'year_of_measurement'},inplace= True)

In [8]:
# read in data, skip rows so that headings become column names
dictxls = pd.ExcelFile('vehicle_fleet_dictionary.xlsx')

# read in data, set the header to be the desired column titles
geo_dict = pd.read_excel(dictxls, sheet_name= 'geo_dictionary', index_col= 0)
geo_name = geo_dict['name'].str.lower().to_list()
country_code = geo_dict['country_code'].astype(str).to_list()
region_code = geo_dict['region_code'].astype(str).to_list()
geo_dict.head(2)

,name,country_code,region_code,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
id,,,,,,,
282,World,1,1,NaN,NaN,NaN,NaN
314,all countries,1,1,NaN,NaN,NaN,NaN


In [9]:
# lowercase the strings in the geo column and replace them with the 3 number codes
df['geo']= df.geo.str.lower()
df.replace(to_replace= geo_name, value= country_code, inplace=True )
df

,year_of_measurement,motor_energy,geo,UNIT,Value,footnote
0,2012,Total,56,NR,5444000,NaN
1,2012,Total,100,NR,2807000,NaN
2,2012,Total,203,NR,4706000,d
3,2012,Total,208,NR,:,NaN
4,2012,Total,276,NR,43431000,NaN
...,...,...,...,...,...,...
3803,2018,Other,578,NR,1008,NaN
3804,2018,Other,756,NR,3700,NaN
3805,2018,Other,807,NR,0,NaN
3806,2018,Other,792,NR,38815,d


In [10]:
# MOTOR ENERGY code replacement
# read in data, skip rows so that headings become column names
dictxls = pd.ExcelFile('vehicle_fleet_dictionary.xlsx')
# assemble drive train code dictionary
me_dict = pd.read_excel(dictxls, sheet_name= 'fuel_types', index_col= 0)
me_eurostat = me_dict['eurostat'].to_list()
me_code = me_dict['dt_code'].to_list()

df['motor_energy'].replace(to_replace=me_eurostat, value=me_code, inplace=True )
#find codes which did not match 
df.loc[~df['motor_energy'].isin(me_code)]

,year_of_measurement,motor_energy,geo,UNIT,Value,footnote


In [11]:
#FLAGS and FOOTNOTES
# read in data, skip rows so that headings become column names
dictxls = pd.ExcelFile('vehicle_fleet_dictionary.xlsx')
# assemble drive train code dictionary
flag_dict = pd.read_excel(dictxls, sheet_name= 'footnote', index_col= 0)
flag_eurostat = flag_dict['flag_eurostat'].to_list()
flag_code = flag_dict['flag_code'].to_list()
flag_dict.tail(2)

df['footnote'].replace(to_replace=flag_code, value=flag_eurostat, inplace=True )
#find codes which did not match 
df.loc[~df['footnote'].isin(flag_eurostat)]

,year_of_measurement,motor_energy,geo,UNIT,Value,footnote
0,2012,all,56,NR,5444000,NaN
1,2012,all,100,NR,2807000,NaN
3,2012,all,208,NR,:,NaN
4,2012,all,276,NR,43431000,NaN
5,2012,all,233,NR,602100,NaN
...,...,...,...,...,...,...
3802,2018,ICE,438,NR,0,NaN
3803,2018,ICE,578,NR,1008,NaN
3804,2018,ICE,756,NR,3700,NaN
3805,2018,ICE,807,NR,0,NaN


In [12]:
'''
what does the missing data mean? Does it mean there were no vehicles of that type? or does it mean that there is no data recorded for that type? """
'''

df=df[df['Value']!= ':']
df

,year_of_measurement,motor_energy,geo,UNIT,Value,footnote
0,2012,all,56,NR,5444000,NaN
1,2012,all,100,NR,2807000,NaN
2,2012,all,203,NR,4706000,"definition differs, see eurostat metadata"
4,2012,all,276,NR,43431000,NaN
5,2012,all,233,NR,602100,NaN
...,...,...,...,...,...,...
3803,2018,ICE,578,NR,1008,NaN
3804,2018,ICE,756,NR,3700,NaN
3805,2018,ICE,807,NR,0,NaN
3806,2018,ICE,792,NR,38815,"definition differs, see eurostat metadata"


## structuring the data into format of datastructure


In [13]:
#rename the data frame to df

# add in a column of indexes
df.index = notebook_id + df.index.astype(str).str.zfill(9)

In [14]:
#rename or add all necessary columns
# df.loc[:,'year_of_measurement']= df.loc[:,'year']
# date of measurement is concatonated to year of measurement to achieve desired format
df.loc[:,'date_of_measurement']=''

/Users/ericyoung/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/ericyoung/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [15]:
# df.loc[:,'geo']= 
df.loc[:,'process']= 'r'
df.loc[:,'vehicle_class']= 'EUM1'
df.loc[:,'vehicle_segment']='all'
df.loc[:,'motor_energy']= df.motor_energy
df.loc[:,'model_year']= 'all'
df.loc[:,'year_of_first_registraion']=''
df.loc[:,'value']= df['Value']
df.loc[:,'unit']= 'nr'
df.loc[:,'source']= nb_data_source
df.loc[:,'accessed']= '2020-07-03'
df.loc[:,'notebook']= nb_name
# df.loc[:,'footnote']= ''

In [16]:
# create a finalized dataframe for output with columns in arranged order
heading_list= ['year_of_measurement','date_of_measurement','geo','process','vehicle_class','vehicle_segment','motor_energy','model_year','year_of_first_registraion','value','unit','source','accessed','notebook','footnote']
df_out= df[heading_list]

df_out

,year_of_measurement,date_of_measurement,geo,process,vehicle_class,vehicle_segment,motor_energy,model_year,year_of_first_registraion,value,unit,source,accessed,notebook,footnote
004000000000,2012,,56,r,EUM1,all,all,all,,5444000,nr,EUROSTAT,2020-07-03,004_s_EU_2012_2018_cl(pc)_me_eurostat.ipynb,NaN
004000000001,2012,,100,r,EUM1,all,all,all,,2807000,nr,EUROSTAT,2020-07-03,004_s_EU_2012_2018_cl(pc)_me_eurostat.ipynb,NaN
004000000002,2012,,203,r,EUM1,all,all,all,,4706000,nr,EUROSTAT,2020-07-03,004_s_EU_2012_2018_cl(pc)_me_eurostat.ipynb,"definition differs, see eurostat metadata"
004000000004,2012,,276,r,EUM1,all,all,all,,43431000,nr,EUROSTAT,2020-07-03,004_s_EU_2012_2018_cl(pc)_me_eurostat.ipynb,NaN
004000000005,2012,,233,r,EUM1,all,all,all,,602100,nr,EUROSTAT,2020-07-03,004_s_EU_2012_2018_cl(pc)_me_eurostat.ipynb,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
004000003803,2018,,578,r,EUM1,all,ICE,all,,1008,nr,EUROSTAT,2020-07-03,004_s_EU_2012_2018_cl(pc)_me_eurostat.ipynb,NaN
004000003804,2018,,756,r,EUM1,all,ICE,all,,3700,nr,EUROSTAT,2020-07-03,004_s_EU_2012_2018_cl(pc)_me_eurostat.ipynb,NaN
004000003805,2018,,807,r,EUM1,all,ICE,all,,0,nr,EUROSTAT,2020-07-03,004_s_EU_2012_2018_cl(pc)_me_eurostat.ipynb,NaN
004000003806,2018,,792,r,EUM1,all,ICE,all,,38815,nr,EUROSTAT,2020-07-03,004_s_EU_2012_2018_cl(pc)_me_eurostat.ipynb,"definition differs, see eurostat metadata"


In [17]:
# write an output file named according to notebook id with relevant data and metadata
writer = pd.ExcelWriter(nb_output_workbook, engine='xlsxwriter')
df_out.to_excel(writer, sheet_name='data', merge_cells=False)
notebook_metadata.to_excel(writer, sheet_name= 'notebook_metadata')
writer.save()

In [18]:
# add the data from the notebook to the stock dataframe and notebook metadata pickles

import pickle
# load in the stock DataFrame pickle
stock_pickle = pd.read_pickle('stock_df.pickle')

# concatenate the out_df to the stock_pickle and remove any duplicate rows
stock_df = pd.concat([stock_pickle,df_out], sort=False).drop_duplicates()

# write the updated stock dataframe to pickle
stock_df.to_pickle('stock_df.pickle')

# repeat the process for the metadata
metadata_pickle = pd.read_pickle('metadata_df.pickle')

metadata_df = pd.concat([metadata_pickle, notebook_metadata], sort=False).drop_duplicates()

metadata_df.to_pickle('metadata_df.pickle')

In [19]:
# update the stock metadata 
writer = pd.ExcelWriter('vehicle_fleet_stock.xlsx', engine='xlsxwriter')
stock_df.to_excel(writer, sheet_name='data', merge_cells=False)
metadata_df.to_excel(writer, sheet_name= 'notebook_metadata')
writer.save()